## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
import random
import time
import config
import sys
from datetime import datetime
from citipy import citipy 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Palmer,US,42.1584,-72.3287,-0.77,72,100,1.54,overcast clouds,2022-02-13 09:59:33
1,1,Rio Grande,BR,-32.0350,-52.0986,17.75,93,99,6.07,overcast clouds,2022-02-13 09:59:35
2,2,Kapaa,US,22.0752,-159.3190,22.18,78,75,5.66,broken clouds,2022-02-13 09:59:36
3,3,Bilma,NE,18.6853,12.9164,23.92,10,0,4.80,clear sky,2022-02-13 09:59:37
4,4,Waipawa,NZ,-41.4122,175.5153,12.35,86,100,4.62,overcast clouds,2022-02-13 09:59:39
5,5,Ushuaia,AR,-54.8000,-68.3000,8.81,61,75,4.63,broken clouds,2022-02-13 09:57:13
6,6,Mayahi,NE,13.9553,7.6712,26.41,9,0,6.72,clear sky,2022-02-13 09:59:41
7,7,Bandarbeyla,SO,9.4942,50.8122,26.41,62,66,7.45,broken clouds,2022-02-13 09:59:44
8,8,Guerrero Negro,MX,27.9769,-114.0611,17.18,59,100,1.50,overcast clouds,2022-02-13 09:59:45
9,9,Beckley,US,37.7782,-81.1882,-0.98,95,100,2.57,light snow,2022-02-13 09:55:24


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your preferable minimum temperature for your trip in celcius?"))
max_temp = float(input("What is your preferable maximum temperature for your trip in celcius?"))

What is your preferable minimum temperature for your trip in celcius?20
What is your preferable maximum temperature for your trip in celcius?30


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp(C)"] >= min_temp) & (city_data_df["Max Temp(C)"] <= max_temp)]
     
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Kapaa,US,22.0752,-159.3190,22.18,78,75,5.66,broken clouds,2022-02-13 09:59:36
3,3,Bilma,NE,18.6853,12.9164,23.92,10,0,4.80,clear sky,2022-02-13 09:59:37
6,6,Mayahi,NE,13.9553,7.6712,26.41,9,0,6.72,clear sky,2022-02-13 09:59:41
7,7,Bandarbeyla,SO,9.4942,50.8122,26.41,62,66,7.45,broken clouds,2022-02-13 09:59:44
11,11,Middelburg,ZA,-25.7751,29.4648,29.49,17,94,1.72,overcast clouds,2022-02-13 09:59:49


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp(C)            0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Kapaa,US,22.0752,-159.3190,22.18,78,75,5.66,broken clouds,2022-02-13 09:59:36
3,3,Bilma,NE,18.6853,12.9164,23.92,10,0,4.80,clear sky,2022-02-13 09:59:37
6,6,Mayahi,NE,13.9553,7.6712,26.41,9,0,6.72,clear sky,2022-02-13 09:59:41
7,7,Bandarbeyla,SO,9.4942,50.8122,26.41,62,66,7.45,broken clouds,2022-02-13 09:59:44
11,11,Middelburg,ZA,-25.7751,29.4648,29.49,17,94,1.72,overcast clouds,2022-02-13 09:59:49


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp(C)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp(C),Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,22.18,broken clouds,22.0752,-159.3190,
3,Bilma,NE,23.92,clear sky,18.6853,12.9164,
6,Mayahi,NE,26.41,clear sky,13.9553,7.6712,
7,Bandarbeyla,SO,26.41,broken clouds,9.4942,50.8122,
11,Middelburg,ZA,29.49,overcast clouds,-25.7751,29.4648,
12,Rikitea,PF,26.10,clear sky,-23.1203,-134.9692,
15,Puerto Ayora,EC,23.08,broken clouds,-0.7393,-90.3518,
19,Geraldton,AU,27.23,clear sky,-28.7667,114.6000,
20,Marawi,PH,22.03,light rain,7.9986,124.2928,
21,Adiake,CI,29.77,light rain,5.2863,-3.3040,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        
    try:    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found ....skipping.") 

Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head()

,City,Country,Max Temp(C),Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,22.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Bilma,NE,23.92,clear sky,18.6853,12.9164,Expeditions Ténére voyages
7,Bandarbeyla,SO,26.41,broken clouds,9.4942,50.8122,JABIR HOTEL
11,Middelburg,ZA,29.49,overcast clouds,-25.7751,29.4648,Oregon Place Guest House
12,Rikitea,PF,26.10,clear sky,-23.1203,-134.9692,People ThankYou


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp(C)} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, -80), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))